Retrieve experiment results

In [36]:
import json
from os.path import join
import os

import pandas as pd

from cogspaces.pipeline import get_output_dir

from json import JSONDecodeError

basedir_ids = [32]
basedirs = [join(get_output_dir(), 'predict_multi', str(_id), 'run') for _id in basedir_ids]
res_list = []
for basedir in basedirs:
    for exp_dir in os.listdir(basedir):
        exp_dir = join(basedir, exp_dir)
        try:
            config = json.load(open(join(exp_dir, 'config.json'), 'r'))
            info = json.load(open(join(exp_dir, 'info.json'), 'r'))
        except (JSONDecodeError, FileNotFoundError):
            continue
        datasets = config['datasets']
        datasets = '__'.join(datasets)
        seed = config['seed']
        model = config['model']
        source = config['source']
        score = info['score']
        rank = info['rank']
        n_components = config['n_components']
        latent_dropout_rate = config['latent_dropout_rate']
        normalized = config['with_std']
        res = {'datasets': datasets, 'n_components': n_components, 'latent_dropout_rate': latent_dropout_rate, 'seed': seed, 'rank': rank,
               'source': source, 'normalized': normalized,
              'model': model}
        for key, value in score.items():
            res[key] = value
        if basedir == join(get_output_dir(), 'predict_multi', '11', 'run') and model == 'logistic':
            continue
        res_list.append(res)
res = pd.DataFrame(res_list)

Compare results with trace norm, with and without transfer

In [37]:
df_agg = res.groupby(by=['normalized', 'source', 'model', 'datasets', 'alpha', 'beta']).aggregate(['mean', 'std'])

df_agg = df_agg.fillna(0)


dfs = {}
for dataset in ['brainomics', 'archi']:
    this_df = df_agg.query("datasets in ['%s__hcp', '%s']" % (dataset, dataset))
    test_mean = sum(this_df[('test_%s' % dataset, 'mean')] for dataset in [dataset, 'hcp']) / 2
    this_df = this_df.assign(test_mean=test_mean)
    indices = this_df[('test_%s' % dataset, 'mean')].groupby(level=['normalized', 'source', 'model', 'datasets']).aggregate('idxmax')
    this_df = this_df.loc[indices.values]
    dfs[dataset] = this_df

KeyError: 'alpha'

Compare results with 3 datasets

In [38]:
this_df

rank  \
                                                                                                                 mean   
normalized source          model      datasets               n_components latent_dropout_rate input_dropout_rate        
True       hcp_rs_positive non_convex brainomics__archi__hcp 200          0.8                 0.25                 71   

                                                                                                                       \
                                                                                                                  std   
normalized source          model      datasets               n_components latent_dropout_rate input_dropout_rate        
True       hcp_rs_positive non_convex brainomics__archi__hcp 200          0.8                 0.25                0.0   

                                                                                                                        seed  \
                                                                                                                        mean   
normalized source          model      datasets               n_components latent_dropout_rate input_dropout_rate               
True       hcp_rs_positive non_convex brainomics__archi__hcp 200          0.8                 0.25                2330081002   

                                                                                                                                \
                                                                                                                           std   
normalized source          model      datasets               n_components latent_dropout_rate input_dropout_rate                 
True       hcp_rs_positive non_convex brainomics__archi__hcp 200          0.8                 0.25                1.242420e+09   

                                                                                                                 test_archi  \
                                                                                                                       mean   
normalized source          model      datasets               n_components latent_dropout_rate input_dropout_rate              
True       hcp_rs_positive non_convex brainomics__archi__hcp 200          0.8                 0.25                 0.873205   

                                                                                                                            \
                                                                                                                       std   
normalized source          model      datasets               n_components latent_dropout_rate input_dropout_rate             
True       hcp_rs_positive non_convex brainomics__archi__hcp 200          0.8                 0.25                0.013366   

                                                                                                                 test_brainomics  \
                                                                                                                            mean   
normalized source          model      datasets               n_components latent_dropout_rate input_dropout_rate                   
True       hcp_rs_positive non_convex brainomics__archi__hcp 200          0.8                 0.25                      0.912943   

                                                                                                                           \
                                                                                                                      std   
normalized source          model      datasets               n_components latent_dropout_rate input_dropout_rate            
True       hcp_rs_positive non_convex brainomics__archi__hcp 200          0.8                 0.25                0.01332   

                                                                 

In [39]:
indices.values

array([ (True, 'hcp_rs_positive', 'non_convex', 'brainomics__archi__hcp', 200, 0.80000000000000004, 0.25)], dtype=object)

In [40]:
this_df = df_agg.query("datasets == 'brainomics__archi__hcp'")
test_mean = sum(this_df[('test_%s' % dataset, 'mean')] for dataset in ['brainomics', 'archi', 'hcp']) / 3
this_df = this_df.assign(test_mean=test_mean)
indices = this_df[('test_mean', 'mean')].groupby(level=['normalized', 'source', 'model', 'datasets']).aggregate('idxmax')
this_df = this_df.loc[indices.values]
dfs['archi+brainomics+hcp'] = this_df

In [41]:
result_df = pd.concat(dfs.values(), keys=dfs.keys(), names=['target'])

In [42]:
result_df

rank  \
                                                                                                                                             mean   
target               normalized source                 model      datasets               n_components latent_dropout_rate input_dropout_rate        
brainomics           True       hcp_rs_positive        non_convex brainomics__hcp        200          0.95                0.25                 41   
                                hcp_rs_positive_single non_convex brainomics__hcp        200          0.90                0.25                 41   
archi                True       hcp_rs_positive        non_convex archi__hcp             200          0.90                0.25                 53   
                                hcp_rs_positive_single non_convex archi__hcp             200          0.90                0.25                 53   
archi+brainomics+hcp True       hcp_rs_positive        non_convex brainomics__archi__hcp 200          0.80                0.25                 71   

                                                                                                                                                   \
                                                                                                                                              std   
target               normalized source                 model      datasets               n_components latent_dropout_rate input_dropout_rate        
brainomics           True       hcp_rs_positive        non_convex brainomics__hcp        200          0.95                0.25                0.0   
                                hcp_rs_positive_single non_convex brainomics__hcp        200          0.90                0.25                0.0   
archi                True       hcp_rs_positive        non_convex archi__hcp             200          0.90                0.25                0.0   
                                hcp_rs_positive_single non_convex archi__hcp             200          0.90                0.25                0.0   
archi+brainomics+hcp True       hcp_rs_positive        non_convex brainomics__archi__hcp 200          0.80                0.25                0.0   

                                                                                                                                                    seed  \
                                                                                                                                                    mean   
target               normalized source                 model      datasets               n_components latent_dropout_rate input_dropout_rate               
brainomics           True       hcp_rs_positive        non_convex brainomics__hcp        200          0.95                0.25                2330081002   
                                hcp_rs_positive_single non_convex brainomics__hcp        200          0.90                0.25                2649957291   
archi                True       hcp_rs_positive        non_convex archi__hcp             200          0.90                0.25                2330081002   
                                hcp_rs_positive_single non_convex archi__hcp             200          0.90                0.25                2330081002   
archi+brainomics+hcp True       hcp_rs_positive        non_convex brainomics__archi__hcp 200          0.80                0.25                2330081002   

                                                                                                                                                            \
                                                                                                                                                       std   
target               normalized source                 model      datasets               n_components latent_dropout_rate input_dropout_rate                 
brainomics           True       

In [43]:
df_agg.query("datasets == 'brainomics__archi__hcp'")

rank  \
                                                                                                                 mean   
normalized source          model      datasets               n_components latent_dropout_rate input_dropout_rate        
True       hcp_rs_positive non_convex brainomics__archi__hcp 200          0.80                0.25                 71   
                                                                                              0.50                 71   
                                                                          0.90                0.25                 71   
                                                                                              0.50                 71   
                                                                          0.95                0.25                 71   
                                                                                              0.50                 71   

                                                                                                                       \
                                                                                                                  std   
normalized source          model      datasets               n_components latent_dropout_rate input_dropout_rate        
True       hcp_rs_positive non_convex brainomics__archi__hcp 200          0.80                0.25                0.0   
                                                                                              0.50                0.0   
                                                                          0.90                0.25                0.0   
                                                                                              0.50                0.0   
                                                                          0.95                0.25                0.0   
                                                                                              0.50                0.0   

                                                                                                                        seed  \
                                                                                                                        mean   
normalized source          model      datasets               n_components latent_dropout_rate input_dropout_rate               
True       hcp_rs_positive non_convex brainomics__archi__hcp 200          0.80                0.25                2330081002   
                                                                                              0.50                2330081002   
                                                                          0.90                0.25                2330081002   
                                                                                              0.50                2330081002   
                                                                          0.95                0.25                2330081002   
                                                                                              0.50                2330081002   

                                                                                                                                \
                                                                                                                           std   
normalized source          model      datasets               n_components latent_dropout_rate input_dropout_rate                 
True       hcp_rs_positive non_convex brainomics__archi__hcp 200          0.80                0.25                1.242420e+09   
                                                                                              0.50                1.242420e+09   
                                                                          0.90                0.25                1.242420e+09   


In [44]:
df

NameError: name 'df' is not defined